In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import dedupe
import time
import os
import joblib

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
gt_train = pd.read_csv("../datasets/ground_truth/GT_train/train.csv")
gt_val = pd.read_csv("../datasets/ground_truth/GT_train/val.csv")
gt_test = pd.read_csv("../datasets/ground_truth/GT_train/test.csv")

In [4]:
df_unificato = pd.read_csv("../datasets/mediated_schema/mediated_schema_normalized.csv", dtype={'id_source_vehicles': 'object'})

/tmp/ipykernel_15471/1061716945.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unificato = pd.read_csv("../datasets/mediated_schema/mediated_schema_normalized.csv", dtype={'id_source_vehicles': 'object'})


In [5]:
df_unificato.drop(columns=['vin', 'description'], inplace=True)

In [6]:
gt_train.drop(columns=['description_A', 'description_B'], inplace=True)
gt_val.drop(columns=['description_A', 'description_B'], inplace=True)
gt_test.drop(columns=['description_A', 'description_B'], inplace=True)

In [7]:
# 1. Creiamo la colonna id_unificato combinando le due sorgenti
# Usiamo fillna() perché abbiamo garantito che dove manca uno c'è l'altro
df_unificato['id_unificato'] = (
    df_unificato['id_source_vehicles']
    .fillna(df_unificato['id_source_used_cars'])
)

# 2. Impostiamo l'id_unificato come INDICE del DataFrame
# Questo è il requisito fondamentale per compare.compute()
df_unificato = df_unificato.set_index('id_unificato')

In [8]:
df_unificato.head()

,id_source_vehicles,id_source_used_cars,location,price,year,manufacturer,model,cylinders,fuel_type,mileage,transmission,traction,body_type,main_color,latitude,longitude,pubblication_date
id_unificato,,,,,,,,,,,,,,,,,
7316814884,7316814884,NaN,auburn,33590.0,2014.0,gmc,sierra 1500 crew cab slt,8 cylinders,gas,57923.0,other,NaN,pickup,white,32.590,-85.4800,2021-05-04T12:31:18-0500
7316814758,7316814758,NaN,auburn,22590.0,2010.0,chevrolet,silverado 1500,8 cylinders,gas,71229.0,other,NaN,pickup,blue,32.590,-85.4800,2021-05-04T12:31:08-0500
7316814989,7316814989,NaN,auburn,39590.0,2020.0,chevrolet,silverado 1500 crew,8 cylinders,gas,19160.0,other,NaN,pickup,red,32.590,-85.4800,2021-05-04T12:31:25-0500
7316743432,7316743432,NaN,auburn,30990.0,2017.0,toyota,tundra double cab sr,8 cylinders,gas,41124.0,other,NaN,pickup,red,32.590,-85.4800,2021-05-04T10:41:31-0500
7316356412,7316356412,NaN,auburn,15000.0,2013.0,ford,f150 xlt,6 cylinders,gas,128000.0,automatic,rwd,truck,black,32.592,-85.5189,2021-05-03T14:02:03-0500


In [9]:
df_unificato.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3255536 entries, 7316814884 to S2_3000039
Data columns (total 17 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_source_vehicles   object 
 1   id_source_used_cars  object 
 2   location             object 
 3   price                float64
 4   year                 float64
 5   manufacturer         object 
 6   model                object 
 7   cylinders            object 
 8   fuel_type            object 
 9   mileage              float64
 10  transmission         object 
 11  traction             object 
 12  body_type            object 
 13  main_color           object 
 14  latitude             float64
 15  longitude            float64
 16  pubblication_date    object 
dtypes: float64(5), object(12)
memory usage: 447.1+ MB


In [10]:
# 2. Trasformazione Year, Latitude e Longitude
# Usiamo una funzione per evitare che 2020 diventi "2020.0"
def to_clean_string(val):
    if pd.isnull(val):
        return ""
    # Se è un numero (float o int), lo trasformiamo in intero se non ha decimali significativi
    if isinstance(val, (float, int)):
        if val == int(val):
            return str(int(val))
    return str(val).strip()

cols_to_fix = ['year', 'latitude', 'longitude']

for col in cols_to_fix:
    df_unificato[col] = df_unificato[col].apply(to_clean_string)

In [11]:
# 3. Verifica rapida dei tipi
print(df_unificato[cols_to_fix].dtypes)
print(df_unificato[cols_to_fix].head())

year         object
latitude     object
longitude    object
dtype: object
              year latitude longitude
id_unificato                         
7316814884    2014    32.59    -85.48
7316814758    2010    32.59    -85.48
7316814989    2020    32.59    -85.48
7316743432    2017    32.59    -85.48
7316356412    2013   32.592  -85.5189


In [12]:
# 1. Identifichiamo le colonne per tipologia in base ai tuoi 'fields'
cols_string = ['location', 'manufacturer', 'model', 'cylinders', 
               'transmission', 'fuel_type', 'year', 'latitude', 'longitude',
               'traction', 'body_type', 'main_color']

cols_numeric = ['price', 'mileage']

# 2. Pulizia Colonne Testuali (Object)
for col in cols_string:
    # Convertiamo tutto in stringa, ma trasformiamo i 'nan' testuali in stringhe vuote
    # Dedupe 3.0 preferisce stringhe vuote o None per i campi String
    df_unificato[col] = df_unificato[col].astype(str).replace(['nan', 'None', 'NaN'], '')

# 3. Pulizia Colonne Numeriche (float64)
for col in cols_numeric:
    # Per i numeri, Dedupe 3.0 richiede l'oggetto None di Python per i mancanti
    # Questo permette a 'has_missing': True di funzionare correttamente
    df_unificato[col] = df_unificato[col].replace({np.nan: None})

In [13]:
# --- STEP C: CAMPIONAMENTO STRATIFICATO ---
# 1. ID per Addestramento e Validazione (Dalla tua Ground Truth)
ids_gt_train_val = set(gt_train['id_A']) | set(gt_train['id_B']) | \
                   set(gt_val['id_A']) | set(gt_val['id_B'])

# 2. Campione casuale dal dataset per le statistiche di Blocking (es. 30.000 record)
# Usiamo solo record che NON sono nel Test Set per purismo metodologico
df_no_test = df_unificato.loc[~df_unificato.index.isin(set(gt_test['id_A']) | set(gt_test['id_B']))]
ids_sample = set(df_no_test.sample(n=min(5000, len(df_no_test)), random_state=42).index)

# 3. Unione ID e creazione dizionario
ids_finali = ids_gt_train_val | ids_sample
data_train_only = df_unificato.loc[list(ids_finali), cols_string + cols_numeric].to_dict('index')

# LIBERA RAM: Se il PC è al limite, puoi eliminare df_unificato qui (avendolo salvato su CSV prima)
# del df_unificato
# gc.collect()

print(f"Dizionario creato con {len(data_train_only)} record.")

Dizionario creato con 26358 record.


In [14]:
# Definiamo i campi utilizzando i nuovi oggetti Variable di Dedupe 3.0
fields = [
    dedupe.variables.String("manufacturer"),
    dedupe.variables.String("model"),
    dedupe.variables.String("year"),
    dedupe.variables.Categorical(
        "transmission", 
        categories=['other', 'automatic', 'manual', 'cvt', 'dual clutch'],
        has_missing=True
    ),
    dedupe.variables.Categorical(
        "fuel_type", 
        categories=['gas', 'other', 'diesel', 'hybrid', 'electric',
                   'biodiesel', 'flex fuel vehicle', 'compressed natural gas', 'propane'],
        has_missing=True
    ),
    dedupe.variables.Price("price"),
    dedupe.variables.Price("mileage", has_missing=True),
    dedupe.variables.String("location"),
    dedupe.variables.String("cylinders", has_missing=True),
    dedupe.variables.Categorical(
        "traction", 
        categories=['rwd', '4wd', 'fwd', 'awd', '4x2'], 
        has_missing=True
    ),
    dedupe.variables.String("body_type", has_missing=True),
    dedupe.variables.String("main_color", has_missing=True),
    dedupe.variables.String("latitude", has_missing=True),
    dedupe.variables.String("longitude", has_missing=True)
]

In [ ]:
# 1. Inizializza con Dedupe (per ricerca interna + linkage)
linker = dedupe.Dedupe(fields)

# 2. Prepara gli esempi dalla Ground Truth
matches = []
distinct = []

for _, row in gt_train.iterrows():
    # Recuperiamo i dati completi dai record usando gli ID della GT
    if row['id_A'] in data_train_only and row['id_B'] in data_train_only:
        pair = (data_train_only[row['id_A']], data_train_only[row['id_B']])
        if row['label'] == 1:
            matches.append(pair)
        else:
            distinct.append(pair)

# 3. Passa gli esempi
linker.mark_pairs({'match': matches, 'distinct': distinct})

# 4. PREPARE_TRAINING (Passando solo data_d una volta)
print("Preparazione dell'addestramento")
linker.prepare_training(
    data=data_train_only, 
    sample_size=10, 
    blocked_proportion=0.5
) 

# 5. TRAIN
print("Addestramento Dedupe in corso")
linker.train()

Preparazione dell'addestramento


In [15]:
# 1. Inizializza con Dedupe (per ricerca interna + linkage)
linker = dedupe.Dedupe(fields)

# 2. Prepara gli esempi dalla Ground Truth
matches = []
distinct = []

for _, row in gt_train.iterrows():
    # Recuperiamo i dati completi dai record usando gli ID della GT
    if row['id_A'] in data_train_only and row['id_B'] in data_train_only:
        pair = (data_train_only[row['id_A']], data_train_only[row['id_B']])
        if row['label'] == 1:
            matches.append(pair)
        else:
            distinct.append(pair)

# 3. Passa gli esempi
linker.mark_pairs({'match': matches, 'distinct': distinct})

# 4. TRAIN
print("Addestramento Dedupe in corso")
linker.train()

Addestramento Dedupe in corso


AssertionError: Please initialize with the prepare_training method